In [51]:
import os
import matplotlib.pyplot as plt
from skimage import io
import cv2

In [52]:
def apex_from_mask(mask):
    height, width = mask.shape

    # Calculate the middle row index
    middle_row = height // 2

   # Initialize variables to track the indices of -1 values
    first_neg_one_idx = None
    last_neg_one_idx = None

    # Iterate over the rows from the middle row to the bottom row
    for y in range(middle_row, height):
        row = mask[y, :]
        # Check if there are any -1 values in the row
        if 1 in row:
            # If this is the first row with -1 values, record the indices and the line index
            if first_neg_one_idx is None:
                first_neg_one_idx = np.where(row == 1)[0][0]
                last_neg_one_idx = np.where(row == 1)[0][-1]
                line_idx = y
                center_idx = (first_neg_one_idx + last_neg_one_idx) // 2
                return (line_idx, center_idx )
      
    for y in range(0, middle_row):
        row = mask[y, :]
        # Check if there are any -1 values in the row
        if 1 in row:
            # If this is the first row with -1 values, record the indices and the line index
            if first_neg_one_idx is None:
                first_neg_one_idx = np.where(row == 1)[0][0]
                last_neg_one_idx = np.where(row == 1)[0][-1]
                line_idx = y
                center_idx = (first_neg_one_idx + last_neg_one_idx) // 2
                return (line_idx, center_idx )

    return None

def compute_mask(mask):
    res = mask.copy()
    res = np.where(res >= 0.95, 1, res)
    res = np.where(res < 0.95, 0, res)
    return res

In [53]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import numpy as np

class MeanMetricWrapper(tf.keras.metrics.Mean):
    def __init__(self, name='mean_metric', dtype=None):
        super(MeanMetricWrapper, self).__init__(name=name, dtype=dtype)

    def get_config(self):
        return {'name': self.name}

    @classmethod
    def from_config(cls, config):
        return cls(name=config['name'])

model = tf.keras.models.load_model("first_full_model", custom_objects={'MeanMetricWrapper': MeanMetricWrapper})

In [54]:
path = "../../../roadfollower/scripts/data/datasets/final_apex/test/apex/"

distances = []
distances_y = []

for item in tqdm([path+itm for itm in os.listdir(path)]):
    img_dem = (cv2.resize(io.imread(item)*1./255, (224,224)))
    img_dem = np.expand_dims(img_dem, axis=0)

    res_dem = model.predict(img_dem)[0]
    orange_lines = compute_mask(res_dem[:,:,2])
    white_lines = compute_mask(res_dem[:,:,1])

    if apex_from_mask(orange_lines):
        pred = apex_from_mask(orange_lines)
    else:
        pred = apex_from_mask(white_lines)
    
    true_x = item.split('/')[-1].split('_')[0]
    true_y = item.split('/')[-1].split('_')[1]
    true = (int(true_x), int(true_y))  
    
    distances.append(np.linalg.norm(np.array(true) - np.array(pred)))
    distances_y.append(np.linalg.norm(np.array(true)[1] - np.array(pred)[1]))


  0%|                                                   | 0/204 [00:00<?, ?it/s]

1/1 [==============================] - 1s 525ms/step


  0%|▏                                          | 1/204 [00:00<01:59,  1.70it/s]

1/1 [==============================] - 0s 66ms/step


  1%|▍                                          | 2/204 [00:00<01:00,  3.31it/s]

...

 99%|████████████████████████████████████████▍| 201/204 [00:20<00:00,  9.82it/s]

1/1 [==============================] - 0s 61ms/step


100%|████████████████████████████████████████▊| 203/204 [00:20<00:00,  9.90it/s]

1/1 [==============================] - 0s 59ms/step


100%|█████████████████████████████████████████| 204/204 [00:20<00:00,  9.73it/s]


In [56]:
import pandas as pd 

#Some insights about distances
desc = pd.Series(distances).describe()
for idx in desc.index:
    print(idx, " : ", desc[idx])


#Processing the distances between predicted apex and the real one
print('Pixel errors description on Y:')


#Some insights about distances
desc = pd.Series(distances_y).describe()
for idx in desc.index:
    print(idx, " : ", desc[idx])

count  :  204.0
mean  :  15.663390108588219
std  :  16.46301645657809
min  :  0.0
25%  :  8.200222875501128
50%  :  11.661903789690601
75%  :  18.601075237738275
max  :  171.72361514946044
Pixel errors description on Y:
count  :  204.0
mean  :  8.970588235294118
std  :  13.705887327551103
min  :  0.0
25%  :  3.0
50%  :  6.0
75%  :  11.0
max  :  167.0
